In [1]:
import matplotlib.pyplot as plt
import scipy.constants as sc
import numpy as np
import xtrack as xt
import xcoll as xc
import xobjects as xo
import xpart as xp
from pathlib import Path
import lossmaps as lm
import cernlayoutdb as layout

 If you are on SWAN and did not expect this warning, then restart and load the 'NXCals Pro' software stack.


In [2]:
#No idea what it is doing
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir="/Users/gebruiker/packages/instantclient_19_8")
## cx_Oracle.init_oracle_client(lib_dir="/Users/your_username/Downloads/instantclient_19_8")

cx_Oracle.clientversion()  
print(cx_Oracle.version)
cx_Oracle.clientversion()

8.3.0


(19, 8, 0, 0, 0)

BeamElement.name_associated_aperture in thick line

In [3]:
path_lines = Path('/Users/gebruiker/sps_simulations/injection_lines')
line = xt.Line.from_json(Path(path_lines,'injection_thick_no_ap_with_deferred_expressions.json'))

Loading line from dict:   0%|          | 0/4426 [00:00<?, ?it/s]

Done loading line from dict.           


In [4]:
#Set cavitiy voltages correctly
line['acl.31735'].frequency = 800e6
line['acl.31735'].voltage = 2 * 0.18e6
line['acl.31735'].lag = 180

line['actcse.31632'].frequency = 200e6
line['actcse.31632'].voltage = 4.5e6
line['actcse.31632'].lag = 180

In [5]:
db = layout.DB()
aperture_info_list = db.get_apertures(machine='SPS', version='EYETS 2024-2025')

l_uniques = {'Bend' : np.unique([name[:4] for name in line.get_elements_of_type(xt.Bend)[1]]),
             'Quadrupole': np.unique([name[:4] for name in line.get_elements_of_type(xt.Quadrupole)[1]]),
             'Sextupole': np.unique([name[:4] for name in line.get_elements_of_type(xt.Sextupole)[1]]),
             'Octupole': np.unique([name[:4] for name in line.get_elements_of_type(xt.Octupole)[1]]),
             'Multipole': np.unique([name[:4] for name in line.get_elements_of_type(xt.Multipole)[1]]),
             'Cavity': np.unique([name[:4] for name in line.get_elements_of_type(xt.Cavity)[1]])}

In [6]:
dico_types = {}
for elem in aperture_info_list:
    name = elem[1].lower()
    if any(name.startswith(pattern) for patterns in l_uniques.values() for pattern in patterns):
        if name[:4] not in dico_types:
            dico_types[name[:4]] = [elem[2]]
        else:
            if elem[2] not in dico_types[name[:4]]:
                dico_types[name[:4]].append(elem[2])

In [7]:
aperture_data_per_unique = {}

for key in dico_types:
    if len(dico_types[key])== 1:
        # print(key)
        # print(np.unique([elem[4][1][0] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        # print(np.unique([elem[4][1][1] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        # print(np.unique([elem[4][1][2] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        # print(np.unique([elem[4][1][3] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        # print(np.unique([elem[4][0] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        # print(' ')
        aperture_data_per_unique[key] = {'aper_type' : str(np.unique([elem[4][0] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0]),
                                       'max_x' : float(np.unique([elem[4][1][0] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0]),
                                       'max_y' : float(np.unique([elem[4][1][1] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0]),
                                       'a' : float(np.unique([elem[4][1][2] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0]),
                                       'b' : float(np.unique([elem[4][1][3] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0])}

In [8]:
aperture_data_per_unique['actc'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : 0.065,
                                       'max_y' : 0.065,
                                       'a' : 0.065,
                                       'b' : 0.065}

aperture_data_per_unique['lqsa'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.08,
                                       'max_y' : 0.08,
                                       'a' : None,
                                       'b' : None,
                                       'Note' : 'Simplified to a rectangle'}

aperture_data_per_unique['qe.'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.0661,
                                       'max_y' : 0.0661,
                                       'a' : None,
                                       'b' : None,
                                       'Note' : 'Simplified and approximated'}

aperture_data_per_unique['qecd'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.0661,
                                       'max_y' : 0.0661,
                                       'a' : None,
                                       'b' : None,
                                       'Note' : 'Simplified and approximated'}

aperture_data_per_unique['qfa.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.05378,
                                       'b' : 0.05378,
                                       'Note' : 'Simplified and approximated'}

aperture_data_per_unique['lod.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.0415,
                                       'b' : 0.0415,
                                       'Note' : 'From old file, no CAD in LDB'}

aperture_data_per_unique['adkcv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1,
                                       'Note' : 'no CAD in LDB'}

aperture_data_per_unique['adksv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1,
                                       'Note' : 'no CAD in LDB'}

aperture_data_per_unique['acfca'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1,
                                       'Note' : 'no CAD in LDB'}

aperture_data_per_unique['mdhb'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.36,
                                       'max_y' : 0.118,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mdhd'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.32,
                                       'max_y' : 0.09,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mdhw'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.2,
                                       'max_y' : 0.2,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mdvw'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.2,
                                       'max_y' : 0.2,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mkdh'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.103,
                                       'b' : 0.103,
                                       'Note' : 'mkdhb, no mkdha and even b is weird drawing'}

aperture_data_per_unique['mkdv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.103,
                                       'b' : 0.103}

aperture_data_per_unique['mke.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.103,
                                       'b' : 0.103}

aperture_data_per_unique['mkp'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.103,
                                       'b' : 0.103}

aperture_data_per_unique['mkqv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.08102,
                                       'b' : 0.08102}

aperture_data_per_unique['mplh'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.05,
                                       'max_y' : 0.38617,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mse.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1339,
                                       'b' : 0.1339}

aperture_data_per_unique['mst.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1339,
                                       'b' : 0.1339,
                                       'Note' : 'No cad but its supposed to be one tank'}

aperture_data_per_unique['zs.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.10755,
                                       'b' : 0.10755}


aperture_data_per_unique['acfca'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1}

In [9]:
for key in aperture_data_per_unique:
    aper_type = aperture_data_per_unique[key]['aper_type']
    max_x = aperture_data_per_unique[key]['max_x']
    max_y = aperture_data_per_unique[key]['max_y']
    a = aperture_data_per_unique[key]['a']
    b = aperture_data_per_unique[key]['b']
    
    if aper_type == 'CIRCLE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitEllipse(a=a, b=b)
    elif aper_type == 'ELLIPSE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitEllipse(a=a, b=b)
    elif aper_type == 'RECTANGLE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitRect(min_x=-max_x, max_x=max_x, min_y=-max_y, max_y=max_y)
    elif aper_type == 'RECTELLIPSE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitRectEllipse(max_x=max_x, max_y=max_y, a=a, b=b)
    elif aper_type == 'RACETRACK':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitRacetrack(min_x=-max_x, max_x=max_x, min_y=-max_y, max_y=max_y, a=a, b=b)

In [10]:
# Update insert_names to handle variable-length keys
insert_names = [
    {f'{name}_aper_upstream': index, f'{name}_aper_downstream': index + 1}
    for index, name in enumerate(line.element_names)
    if any(name.lower().startswith(key) for key in aperture_data_per_unique)
]
insert_names = {kk: vv for dct in insert_names for kk, vv in dct.items()}
idxs = list(insert_names.values())
names = list(insert_names.keys())
max_length = max(max(map(len, line.element_names)), max(map(len, names)))
element_names = np.array(line.element_names, dtype=f'<U{max_length}')
names = np.array(names, dtype=f'<U{max_length}')
element_names = np.insert(element_names, idxs, names)

# Update insert_elements to handle variable-length keys
insert_elements = [
    {
        f'{name}_aper_upstream': aperture_data_per_unique[next(key for key in aperture_data_per_unique if name.lower().startswith(key))]['xt_elem'].copy(),
        f'{name}_aper_downstream': aperture_data_per_unique[next(key for key in aperture_data_per_unique if name.lower().startswith(key))]['xt_elem'].copy()
    }
    for name in line.element_names
    if any(name.lower().startswith(key) for key in aperture_data_per_unique)
]
insert_elements = {kk: vv for dct in insert_elements for kk, vv in dct.items()}

# Update line's element names and element dictionary
line.element_names = element_names.tolist()
line.element_dict = {**line.element_dict, **insert_elements}

In [11]:
line.twiss()

TwissTable: 7413 rows, 62 cols
name                                  s             x            px             y            py ...
sps$start                             0             0             0             0             0
begi.10010                            0             0             0             0             0
qf.10010_aper_upstream                0             0             0             0             0
qf.10010                              0             0             0             0             0
qf.10010_aper_downstream          3.085             0             0             0             0
drift_0                           3.085             0             0             0             0
mba.10030_aper_upstream         3.44499             0             0             0             0
mba.10030                       3.44499             0             0             0             0
mba.10030_aper_downstream       9.70501  -2.53268e-16  -8.28345e-17             0             0
drift

In [12]:
slice_factor = 1 
slicer = xt.slicing.Slicer(line,
                   slicing_strategies=[xt.Strategy(slicing=xt.Teapot(1)),
                                              xt.Strategy(slicing=xt.Teapot(2), element_type=xt.Bend),
                                              xt.Strategy(slicing=xt.Teapot(2*slice_factor), element_type=xt.Quadrupole),
                                              xt.Strategy(slicing=xt.Teapot(32*slice_factor), name='mqxa.*'),
                                              xt.Strategy(slicing=xt.Teapot(32*slice_factor), name='mqxb.*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mqxb\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mbrb\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mbrc\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mbrs\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mqwa\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mqwb\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4*slice_factor), name='mqy\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4*slice_factor), name='mqm\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4*slice_factor), name='mqmc\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4*slice_factor), name='mqml\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(2*slice_factor), name='mqtlh\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(2*slice_factor), name='mqtli\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(2*slice_factor), name='mqt\\..*'),
                                              xt.Strategy(slicing=None, element_type=xt.Solenoid),
                                              xt.Strategy(slicing=None, name='qe\\..*'), 
                                              xt.Strategy(slicing=None, name='qfa\\..*'),
                                              xt.Strategy(slicing=None, name='qecd\\..*')])

# Test class

In [10]:
import apertls

In [14]:
line = xt.Line.from_json(Path(path_lines,'injection_thick_no_ap_with_deferred_expressions.json'))

Loading line from dict:   0%|          | 0/4426 [00:00<?, ?it/s]

Done loading line from dict.           


In [18]:
#Set cavitiy voltages correctly
line['acl.31735'].frequency = 800e6
line['acl.31735'].voltage = 2 * 0.18e6
line['acl.31735'].lag = 180

line['actcse.31632'].frequency = 200e6
line['actcse.31632'].voltage = 4.5e6
line['actcse.31632'].lag = 180

In [11]:
line.slice_thick_elements(slicing_strategies=[xt.Strategy(slicing=xt.Teapot(4))])

Slicing line:   0%|          | 0/4426 [00:00<?, ?it/s]

{'sps$start': ['sps$start'],
 'begi.10010': ['begi.10010'],
 'qf.10010': ['qf.10010_entry',
  'drift_qf.10010..0',
  'qf.10010..0',
  'drift_qf.10010..1',
  'qf.10010..1',
  'drift_qf.10010..2',
  'qf.10010..2',
  'drift_qf.10010..3',
  'qf.10010..3',
  'drift_qf.10010..4',
  'qf.10010_exit'],
 'drift_0': ['drift_0'],
 'mba.10030': ['mba.10030_entry',
  'mba.10030..entry_map',
  'drift_mba.10030..0',
  'mba.10030..0',
  'drift_mba.10030..1',
  'mba.10030..1',
  'drift_mba.10030..2',
  'mba.10030..2',
  'drift_mba.10030..3',
  'mba.10030..3',
  'drift_mba.10030..4',
  'mba.10030..exit_map',
  'mba.10030_exit'],
 'drift_1': ['drift_1'],
 'mba.10050': ['mba.10050_entry',
  'mba.10050..entry_map',
  'drift_mba.10050..0',
  'mba.10050..0',
  'drift_mba.10050..1',
  'mba.10050..1',
  'drift_mba.10050..2',
  'mba.10050..2',
  'drift_mba.10050..3',
  'mba.10050..3',
  'drift_mba.10050..4',
  'mba.10050..exit_map',
  'mba.10050_exit'],
 'drift_2': ['drift_2'],
 'mbb.10070': ['mbb.10070_entry',


In [12]:
ap_manager = apertls.ManageApertures(line, machine='SPS', version='EYETS 2024-2025')

In [13]:
ap_manager.associated_apertures_naming(complementary_apertures=aperture_data_per_unique)

In [14]:
ap_manager.install_apertures()

In [15]:
line.check_aperture()

Checking aperture:   0%|          | 0/30544 [00:00<?, ?it/s]

Done checking aperture.           
0 thin elements miss associated aperture (upstream):
[]
0 thick elements miss associated aperture (upstream or downstream):
[]


,s,element_type,name,isthick,isreplica,parent_name,iscollective,element,s_start,s_center,s_end,is_aperture,i_aperture_upstream,s_aperture_upstream,i_aperture_downstream,s_aperture_downstream,misses_aperture_upstream,misses_aperture_downstream,has_aperture_problem
0,0.0000,Marker,sps$start,False,False,None,False,Marker(),0.0000,0.00000,0.0000,False,NaN,NaN,NaN,NaN,False,False,False
1,0.0000,Marker,begi.10010,False,False,None,False,Marker(),0.0000,0.00000,0.0000,False,NaN,NaN,NaN,NaN,False,False,False
2,0.0000,Marker,qf.10010_entry,False,False,None,False,Marker(),0.0000,0.00000,0.0000,False,NaN,NaN,NaN,NaN,False,False,False
3,0.0000,DriftSliceQuadrupole,drift_qf.10010..0,True,False,qf.10010,False,"DriftSliceQuadrupole(parent_name='qf.10010', _...",0.0000,0.15425,0.3085,False,NaN,NaN,NaN,NaN,False,False,False
4,0.3085,LimitEllipse,qf.10010..0_aper_upstream,False,False,None,False,"LimitEllipse(a=0.076, b=0.0192)",0.3085,0.30850,0.3085,True,NaN,NaN,NaN,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30544,6911.1338,Drift,bph.63608,True,False,None,False,Drift(length=0.275),6911.1338,6911.27130,6911.4088,False,NaN,NaN,NaN,NaN,False,False,False
30545,6911.4088,Drift,drift_1879,True,False,None,False,Drift(length=0.095),6911.4088,6911.45630,6911.5038,False,NaN,NaN,NaN,NaN,False,False,False
30546,6911.5038,Marker,end.10010,False,False,None,False,Marker(),6911.5038,6911.50380,6911.5038,False,NaN,NaN,NaN,NaN,False,False,False
30547,6911.5038,Marker,sps$end,False,False,None,False,Marker(),6911.5038,6911.50380,6911.5038,False,NaN,NaN,NaN,NaN,False,False,False


In [16]:
dic_test = ap_manager.find_associated_apertures()